# Astrodynamics
<img src="curiosity-selfy-rock-hall.jpg" width = 600pt/>
<center> Curiosity selfy at "Rock Hall." Image: NASA/JPL

In [1]:
import copy
import math
import numpy as np
import operator

from functools import reduce
from scipy.integrate import ode, solve_bvp
from scipy.interpolate import CubicSpline

import astropy.units as u
from astropy import time
from astropy.coordinates import solar_system_ephemeris, get_body_barycentric_posvel

import poliastro

from poliastro import iod
from poliastro.bodies import Sun
from poliastro.twobody import Orbit

from poliastro.twobody.propagation import propagate

from poliastro.util import time_range

from poliastro.plotting import OrbitPlotter3D
from poliastro.bodies import Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune

from astropy import constants
import astropy.coordinates as coordinates

from astroquery.jplhorizons import Horizons

from plotly.graph_objs import FigureWidget

import plotly.offline as py
import plotly.graph_objs as go

In [2]:
# global constants in km, kg and day units
AU = 1*u.au.to(u.km)
EARTH_MASS = constants.M_earth.value
GRAVITATIONAL_CONSTANT = (constants.G.to(u.km**3/u.kg/u.d**2)).value
GAUSSIAN_CONSTANT = 1.720209895e-2
LIGHT_SPEED = constants.c.to(u.km/u.d).value
PI = math.pi
SUN_LUMINOSITY = constants.L_sun.to(u.kg*u.km**2/u.d**3).value

In [3]:
# setup
py.init_notebook_mode(connected=True)
solar_system_ephemeris.set("jpl")

# number of steps
N = 800

date_launch = time.Time("2011-11-26 15:02", scale="utc")
date_arrival = time.Time("2012-08-06 05:17", scale="utc")
tof = date_arrival - date_launch

print('Time to rendezvous:', tof.to(u.d))

Time to rendezvous: 253.59376157407408 d


In [4]:
def get_state(name, epochs):
    obj = Horizons(id=name, location='500@10', epochs=epochs)
    vec = obj.vectors()
    x = vec['x'].data*AU
    y = vec['y'].data*AU
    z = vec['z'].data*AU
    
    vx = vec['vx'].data*AU
    vy = vec['vy'].data*AU
    vz = vec['vz'].data*AU
    
    state = (coordinates.representation.CartesianRepresentation(x,y,z, unit=u.km), 
             coordinates.representation.CartesianRepresentation(vx,vy,vz, unit=(u.km/u.d)))
    
    return state

In [5]:
def orbit(t, rv):
    r = np.array([i.xyz.value for i in rv])
    return CubicSpline(t, r)

In [6]:
times_vector = time_range(date_launch, end=date_arrival, periods=N)
times_from_launch = np.array([(t - date_launch).value for t in times_vector])

planets_names = ["mercury", "venus", "earth", "moon", "mars", "jupiter", "saturn", "uranus", "neptune","pluto"]
planets_masses_relative_to_earth = [0.0553, 0.815, 1.0, 0.0123, 0.107, 317.8, 95.16, 14.54, 17.15, 0.0022]
planets_diameters = [4879.0, 12104.0, 12756.0, 3475.0, 6792.0, 142984.0, 120536.0, 51118.0, 49528.0, 2370.0]

big3_names = ["ceres", "pallas", "vesta"]
big3_masses_relative_to_earth = [0.00015727872, 3.533e-5, 4.338e-5]
big3_diameters = [946.0, 512.0, 525.4]
# Horizons doesn't accept the dates with hours and min
big3_start_date = '2011-11-25'
big3_stop_date = '2012-08-07'
big3_steps = '1d'
big3_N = time.Time(big3_stop_date, scale="utc") - time.Time(big3_start_date, scale="utc")
big3_times_vector = time_range(big3_start_date, spacing=u.d, periods=int(big3_N.to(u.d).value) + 1)
big3_times_from_start = np.array([(t - time.Time(big3_start_date, scale="utc")).value for t in big3_times_vector])

solar_system_ephemeris.set('jpl')

bodies = dict()
for name in planets_names:
    bodies[name] = dict()
    state = get_body_barycentric_posvel(name, times_vector)
    bodies[name]['state'] = state
    bodies[name]['orbit'] = orbit(times_from_launch, state[0])
    bodies[name]['mass'] = EARTH_MASS*planets_masses_relative_to_earth[planets_names.index(name)]
    bodies[name]['diameter'] = planets_diameters[planets_names.index(name)]

big3_epochs = {'start':big3_start_date, 'stop':big3_stop_date, 'step':big3_steps, 'scale': 'utc'}
for name in big3_names:
    bodies[name] = dict()
    state = get_state(name, big3_epochs)
    bodies[name]['state'] = state
    bodies[name]['orbit'] = orbit(big3_times_from_start, state[0])
    bodies[name]['mass'] = EARTH_MASS*big3_masses_relative_to_earth[big3_names.index(name)]
    bodies[name]['diameter'] = big3_diameters[big3_names.index(name)]

In [7]:
# compute Lambert's transfer orbit
δ = 6.5e3
r0 = bodies['earth']['state'][0][0].xyz
r0 *= (1 + δ/np.linalg.norm(r0.value))
δ = -3396.0
rf = bodies['mars']['state'][0][-1].xyz
rf *= (1 + δ/np.linalg.norm(rf.value))

(va, vb), = iod.lambert(Sun.k, r0, rf, tof)

ss0_trans = Orbit.from_vectors(Sun, r0, va, date_launch)
ssf_trans = Orbit.from_vectors(Sun, rf, vb, date_arrival)
transfer_trajectory = propagate(ss0_trans, time.TimeDelta(times_vector - ss0_trans.epoch))

print("Lambert's satellite initial conditions:\nposition: {}\nvelocity: {}".format(r0, va))
print("\nLambert's satellite final conditions:\nposition: {}\nvelocity: {}".format(rf, vb))

Lambert's satellite initial conditions:
position: [6.46034946e+07 1.21430225e+08 5.26423707e+07] km
velocity: [-29.29053502  14.53148704   5.41612875] km / s

Lambert's satellite final conditions:
position: [-1.29710539e+08 -1.74209401e+08 -7.64068654e+07] km
velocity: [ 17.61593138 -10.99895389  -4.20818503] km / s


In [8]:
colors = dict()
colors["sun"] = "#ffcc00"

colors["mercury"] = "#B0B19C"
colors["venus"] = "#E09B4D"
colors["earth"] = "#405ABC"
colors["moon"] = "#DBDBDB"
colors["mars"] = "#B89779"
colors["jupiter"] = "#FED899"
colors["saturn"] = "#D8C663"
colors["uranus"] = "#1677B2"
colors["neptune"] = "#A5E1ED"
colors["pluto"] = "#471614"

colors["ceres"] = "#A5E1ED"
colors["pallas"] = "#AAAAAA"
colors["vesta"] = "#A6E1FD"

colors["craft"] = "#888888"

fig = FigureWidget()

frame = OrbitPlotter3D(figure=fig, dark=True)

frame.set_attractor(Sun)

for body in bodies:
    frame.plot_trajectory(bodies[body]['state'][0], label=body, color=colors[body])

frame.plot_trajectory(transfer_trajectory, label="Lambert's trajectory", color=colors["craft"])

frame.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

fig.layout.title = "MSL Mission: from Earth to Mars"
fig.layout.paper_bgcolor = "black"
fig

FigureWidget({
    'data': [{'line': {'color': '#B0B19C', 'dash': 'solid', 'width': 5},
              'mode': …

## Boundary Value Problem

In [9]:
sun = {'mass': constants.M_sun.value, 'orbit': lambda t: np.zeros(3)}
craft = {'mass': 1000.0, 'sun bathed area': 1e-5, 'coefficient of solar radiation': 1.0, 'view factor': 1.0}

def newton_gravity(x, mass, x_body):
        """
        Returns the force exerted on this body by the other body.
        """
        tol = 1e-6
        
        dr = x_body - x
        d = np.linalg.norm(dr, 2)

        if d < tol:
            raise ValueError("Collision between {} and the body with mass {}.".format('sat', mass))

        return GRAVITATIONAL_CONSTANT*mass/d/d/d*dr

def conservative_forces(t, x, bodies, sun, obj):
    forces_b = map(lambda name: newton_gravity(x[0:3], bodies[name]['mass'], bodies[name]['orbit'](t)), bodies)
    forces_s = map(lambda name: newton_gravity(np.zeros(3), bodies[name]['mass'], bodies[name]['orbit'](t)), bodies)

    return reduce(operator.add, forces_b) - reduce(operator.add, forces_s) + newton_gravity(x[0:3], sun['mass'], sun['orbit'](t))

def non_conservative_forces(t, x, bodies, sun, obj):
    r = np.array(x[0:3])
    r_len = np.linalg.norm(r, 2)
    v = np.array(x[3:])
    
    # solar radiation pressure
    k = obj['view factor']
    cr = obj['coefficient of solar radiation']
    ar = obj['sun bathed area']
    obj_mass = obj['mass']
    
    fSRP = k*cr*ar/obj_mass*SUN_LUMINOSITY/4/PI/LIGHT_SPEED*r/r_len**3
    
    # relativistic correction
    kGaussSq = GAUSSIAN_CONSTANT**2
    
    fRel = kGaussSq/LIGHT_SPEED**2/r_len**3*(4*kGaussSq*r/r_len - np.dot(v,v)*r + np.dot(v,r)*v)

    return fSRP + fRel

def X(t, x, bodies, sun, obj):
    resultant = conservative_forces(t, x, bodies, sun, obj) + non_conservative_forces(t, x, bodies, sun, obj)
    
    return np.array([x[3], x[4], x[5], resultant[0], resultant[1], resultant[2]])

In [10]:
def bvp_fun(t, x):
    y = np.transpose(x)
    return np.transpose(np.array([X(i, j, bodies, sun, craft) for i,j in zip(t,y)]))

def bvp_bc(ya, yb):
    return np.array([ya[i] - r0[i].value for i in range(3)] + [yb[i] - rf[i].value for i in range(3)])

def bvp_bc_jac(ya, yb):
    return np.array([[1,0,0,0,0,0], [0,1,0,0,0,0], [0,0,1,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]), \
           np.array([[0,0,0,0,0,0], [0,0,0,0,0,0], [0,0,0,0,0,0],[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0]])

In [11]:
final_time = tof.to(u.d).value
v_factor = 1.1
n_inc = 2
x_m = times_from_launch[0::n_inc]
y_m = [[transfer_trajectory[i].data.x.value,
        transfer_trajectory[i].data.y.value,
        transfer_trajectory[i].data.z.value,
        v_factor*transfer_trajectory.v_x[i].to(u.km/u.d).value,
        v_factor*transfer_trajectory.v_y[i].to(u.km/u.d).value,
        v_factor*transfer_trajectory.v_z[i].to(u.km/u.d).value]
        for i in range(0,N,n_inc)]
y_m = np.transpose(np.array(y_m))

bvp_sol = solve_bvp(bvp_fun, bvp_bc, x_m, y_m, bc_jac=bvp_bc_jac, tol=5e-6, max_nodes=1000000, verbose=2)

craft_trajectory= np.array([bvp_sol.sol(t)/AU for t in times_from_launch])

   Iteration    Max residual    Total nodes    Nodes added  
       1          6.74e+02          400            21       
       2          7.81e+01          421            25       
       3          9.69e+00          446            34       
       4          1.56e+00          480            47       
       5          4.55e-01          527            60       
       6          1.21e-02          587            47       
       7          1.91e-03          634            39       
       8          2.04e-04          673            36       
       9          5.24e-05          709            19       
      10          1.40e-05          728             2       
      11          4.98e-06          730             0       
Solved in 11 iterations, number of nodes 730, maximum relative residual 4.98e-06.


In [12]:
def generate_sphere(radius, center, num=50):
    u1 = np.linspace(0, 2 * np.pi, num)
    v1 = u1.copy()
    uu, vv = np.meshgrid(u1, v1)

    x_center, y_center, z_center = center

    xx = x_center + radius * np.cos(uu) * np.sin(vv)
    yy = y_center + radius * np.sin(uu) * np.sin(vv)
    zz = z_center + radius * np.cos(vv)

    return xx, yy, zz

def generate_ellipsoid(a, b, c, center, num=50):
    u1 = np.linspace(0, 2 * np.pi, num)
    v1 = u1.copy()
    uu, vv = np.meshgrid(u1, v1)

    x_center, y_center, z_center = center

    xx = x_center + a * np.cos(uu) * np.sin(vv)
    yy = y_center + b * np.sin(uu) * np.sin(vv)
    zz = z_center + c * np.cos(vv)

    return xx, yy, zz

In [14]:
font_color = '#AAAAAA'
axis_bg_color = "#101010"
grid_color = "#516883"

select_bodies = ['mercury', 'venus', 'earth', 'moon', 'mars', 'ceres', 'pallas', 'vesta']

xx, yy, zz = generate_sphere(0.15, (0.,0.,0.))
trace_sun = go.Mesh3d({
                'x': xx.flatten(), 
                'y': yy.flatten(),
                'z': zz.flatten(), 
                'name': 'sun',
                'alphahull': 0,
                'color': '#F3F32C'
})

# trajectories
data = [trace_sun]*(len(select_bodies)+2)
body_trajectories = dict()
for name in select_bodies:
    body_trajectory = bodies[name]['orbit'](times_from_launch)/AU
    body_trajectories[name] = body_trajectory
    trace_body = go.Scatter3d(x=body_trajectory[:,0], 
                           y=body_trajectory[:,1], 
                           z=body_trajectory[:,2],
                           name=None,
                           showlegend=False, 
                           mode='lines',
                           line=dict(color=colors[name], width=2, dash='dash'))
    data.append(trace_body)

trace_sat = go.Scatter3d(x=craft_trajectory[:,0], 
                           y=craft_trajectory[:,1], 
                           z=craft_trajectory[:,2],
                           name=None,
                           showlegend=False, 
                           mode='lines',
                           line=dict(color=colors['craft'], width=2, dash='dash'))
data.append(trace_sat)

earth_size_factor = 10/bodies['earth']['diameter']
frames = []
for k in range(N):
    k_data = []
    for name in select_bodies:
        body_trajectory = body_trajectories[name]
        body_k_frame = go.Scatter3d(x=[body_trajectory[k,0]], 
                            y=[body_trajectory[k,1]],
                            z=[body_trajectory[k,2]],
                            mode='markers',
                            name=name,
                            marker=dict(color=colors[name], size=bodies[name]['diameter']*earth_size_factor))
        k_data.append(body_k_frame)
    xx, yy, zz = generate_ellipsoid(0.04, 0.04, 0.02, (craft_trajectory[k,0],craft_trajectory[k,1],craft_trajectory[k,2]))
    craft_k_frame = go.Mesh3d({
                'x': xx.flatten(), 
                'y': yy.flatten(),
                'z': zz.flatten(), 
                'name': 'craft',
                'alphahull': 0,
                'color': colors['craft']
    })
    k_data.append(craft_k_frame)
    frames.append(dict(data=k_data))     
    
# camera
distance = 3 * u.km
elev = 30 * u.deg
azim = 260 * u.deg
x_cam = distance * np.cos(elev) * np.cos(azim)
y_cam = distance * np.cos(elev) * np.sin(azim)
z_cam = distance * np.sin(elev)
        
layout = go.Layout(
    paper_bgcolor='#000000',
    transition=dict(duration=0),
    font = dict(color=font_color),
    scene = dict(
                    xaxis = dict(
                         title='x [AU]',
                         backgroundcolor=axis_bg_color,
                         gridcolor=grid_color,
                         showbackground=True,
                         zerolinecolor=grid_color,),
                    yaxis = dict(
                        title='y [AU]',
                        backgroundcolor=axis_bg_color,
                        gridcolor=grid_color,
                        showbackground=True,
                        zerolinecolor=grid_color),
                    zaxis = dict(
                        title='z [AU]',
                        backgroundcolor=axis_bg_color,
                        gridcolor=grid_color,
                        showbackground=True,
                        zerolinecolor=grid_color,),
                    aspectmode="data",
                    camera = dict(eye=dict(
                        x=x_cam.to(u.km).value,
                        y=y_cam.to(u.km).value,
                        z=z_cam.to(u.km).value))
    ),
    title = "MSL Mission: from Earth to Mars",
    updatemenus=[{
                   'buttons': [
                       {'args': [None, {'frame': {'duration': 0, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 0}}],
                        'label': 'Play',
                        'method': 'animate'}
               ],
               'pad': {'r': 10, 't': 87},
               'showactive': False,
               'type': 'buttons'
                }]
)


fig = go.Figure(data=data, layout=layout, frames=frames)

py.plot(fig, filename='animation.html')

'file:///Users/marcelo/Documents/UH/ME 360/Jupyter/lectures/ode/animation.html'